<a href="https://colab.research.google.com/github/pinilDissanayaka/Election-Insight-App/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import PyPDFLoader

In [11]:
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'

In [23]:
constitution_path="/content/constitution.pdf"

loader=PyPDFLoader(file_path=constitution_path)

constitution_data=loader.load()

print(f"{len(constitution_data)} documents are loaded.")
print("="*1000)
print("A sample document is:")
print(constitution_data[0].page_content)

306 documents are loaded.
A sample document is:
   The Constitution  of the Democratic  Socialist  Republic  of Sri Lanka  
 
THE  CONSTITUTION  
 
 
OF THE  
 
 
DEMOCR ATIC SOCIALIST  REPUBLIC 
OF SRI LANKA  
 
 
 
 
(As amended up to 31st October 2022)   
Revised Edition  – 2023  
 
 
 
 
 
 
 
 
 
 
 
 
 
Published by the Parliament  Secretariat  
 
 
  



In [30]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings=HuggingFaceEmbeddings(model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
constitution_data_chunks=SemanticChunker(embeddings=embeddings).split_documents(constitution_data)

print(f"{len(constitution_data_chunks)} chunks are created.")

In [20]:
def createIndex(indexName:str, dimension:int):
  try:
    pinecone=Pinecone()
    if indexName not in pinecone.list_indexes().names():
      pinecone.create_index(
          name=indexName,
          dimension=dimension,
          metric="cosine",
          spec=ServerlessSpec(cloud='aws',
                            region='us-east-1')
      )
      print(f"Created {indexName}")
    else:
      print(f"{indexName} already exists")
    return pinecone.describe_index(indexName)
  except Exception as e:
    print(e.args)

In [21]:
dimension=768


In [26]:
import time

constitution_data_chunks = []
for document in constitution_data:
    chunks = SemanticChunker(embeddings=embeddings).split_documents([document])
    constitution_data_chunks.extend(chunks)
    time.sleep(1)  # Wait for 1 second

print(f"{len(constitution_data_chunks)} chunks are created.")

GoogleGenerativeAIError: Error embedding content: 429 Quota exceeded for quota metric 'Batch Embed Content API requests' and limit 'Batch embed contents request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'project_number:174905663139'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/174905663139"
}
metadata {
  key: "quota_limit"
  value: "BatchEmbedContentsRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "150"
}
metadata {
  key: "quota_location"
  value: "us-east2"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/batch_embed_contents_requests"
}
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quotas/help/request_increase"
}
]

In [28]:
pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 6.5 MB/s eta 0:00:00
